In [7]:
%pip install python-dotenv langchain langchain-upstage langchain-community langchain-text-splitters docx2txt langchain-chroma

Note: you may need to restart the kernel to use updated packages.


## 2. Knowledge Base 구성을 위한 데이터 생성 
- RecursiveCharacterTextSplitter를 활용해서 데이터 chunking 
	- Recursively의 경우 list type
- chunk_size: split된 chunk's max size
- chunk_overlap: 앞 뒤로 나뉘어진 chunk들이 얼마나 겹쳐도 되는지 지정

In [8]:
# from langchain_community.document_loaders import Docx2txtLoader

# loader = Docx2txtLoader('./medical.docx')
# document = loader.load()

In [9]:
# document

In [10]:
# len(document) # 통으로 불러옴

## 3. 문서 쪼개기 (RecursiveCharacterTextSplitter)

단순히 로컬 파일('./medical.docx')을 읽어 텍스트 데이터를 처리하기 때문에 환경변수가 필요하지 않음

In [11]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 200,
)

loader = Docx2txtLoader('./medical.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [13]:
len(document_list)

69

UpstageEmbeddings 객체를 초기화하는 과정에서 API 키나 인증 정보를 필요로 하기 때문에 환경변수 로드가 필요

In [15]:
from dotenv import load_dotenv # 환경변수 로드 
from langchain_upstage import UpstageEmbeddings

load_dotenv()

# Solar Pro will stay free until charging begins on Jan 6, 2025.
embedding = UpstageEmbeddings(model="solar-pro") 

In [ ]:
from langchain_chroma import Chroma

# chroma에 저장
database = Chroma.from_documents(documents=document_list, embedding=embedding, collection_name='chroma-medical', persist_directory="./chroma")

BadRequestError: Error code: 400 - {'error': {'message': 'Requested model is invalid. You can find correct request body form at our API reference page (https://console.upstage.ai/docs/getting-started/overview)', 'type': 'invalid_request_error', 'param': '', 'code': 'invalid_request_body'}}

In [ ]:
query = '환자가 진료 기록 열람을 요청할 수 있는 절차는 무엇인가요?'

# 'k'값을 조정해서 얼마나 많은 데이터를 불러올지 결정
retrieved_docs = database.similarity_search(query, k=3)

In [ ]:
retrieved_docs 